In [4]:
from keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose
from keras.models import Sequential, Model

class STAE(Model):
  def __init__(self):
    super(STAE, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Input(shape=(227, 227, 10, 1)),
      layers.Conv3D(filters= 128, kernel_size= (11, 11, 1), strides= (4, 4, 1), padding='valid', activation='relu'),
      layers.Conv3D(filters=64, kernel_size=(5, 5, 1), strides=(2, 2, 1), padding='valid', activation='relu')])
    
    self.bottleneck = tf.keras.Sequential([
      layers.ConvLSTM2D(filters=64, kernel_size=(3, 3), strides=1, padding='same', dropout=0.4, recurrent_dropout=0.3, return_sequences=True),
      layers.ConvLSTM2D(filters=32, kernel_size=(3, 3), strides=1, padding='same', dropout=0.3, return_sequences=True),
      layers.ConvLSTM2D(filters=64, kernel_size=(3, 3), strides=1, return_sequences=True, padding='same', dropout=0.5)])
    
    self.decoder = tf.keras.Sequential([
      layers.Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='relu'),
      layers.Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='relu')])

  def call(self, x):
    encoded = self.encoder(x)
    reduced = self.bottleneck(encoded) 
    decoded = self.decoder(reduced)
    return decoded

def loadModel():
    model = STAE() 
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

In [5]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np

In [6]:
n_epochs = 1

X_train=np.load('training.npy')
frames=X_train.shape[2]
#Need to make number of frames divisible by 10

frames=frames-frames%10

X_train=X_train[:,:,:frames]
X_train=X_train.reshape(-1,227,227,10)
X_train=np.expand_dims(X_train,axis=4)
Y_train=X_train.copy()


epochs=n_epochs
batch_size=32

ValueError: cannot reshape array of size 57600000 into shape (227,227,10)

In [ ]:
if __name__=="__main__":

	model=loadModel()

	callback_save = ModelCheckpoint("model.h5",
									monitor="mean_squared_error", save_best_only=True)

	callback_early_stopping = EarlyStopping(monitor='val_loss', patience=3)

	print('Model has been loaded')

	model.fit(X_train,Y_train,
			  batch_size=batch_size,
			  epochs=epochs,
			  callbacks = [callback_save,callback_early_stopping]
			  )